In [1]:
import tushare as ts
import pandas as pd
import time

In [2]:
ts.set_token('53cd3b985c649c978160c6ec04bce24f4fbd2ebcb4673e8f2fba9a43')

pro = ts.pro_api()

In [3]:
companys = pro.stock_basic(exchange='', list_status='L', fields='ts_code')

In [4]:
companys.head()

,ts_code
0,000001.SZ
1,000002.SZ
2,000004.SZ
3,000005.SZ
4,000006.SZ


In [5]:
companys = list(companys['ts_code'])

In [6]:
len(companys)

3829

In [7]:
managers = []
i = 0
while i < len(companys):
    com_slice = companys[i:min(i+100, len(companys))]
    com_str = ','.join(com_slice)
    df = pro.stk_managers(ts_code=com_str)
    managers.append(df)
    i += 100
    time.sleep(0.5)

In [8]:
len(managers)

39

In [68]:
managers_df = pd.concat(managers)

In [69]:
managers_df.head()

,ts_code,ann_date,name,gender,lev,title,edu,national,birthday,begin_date,end_date
0,000058.SZ,20190925,徐腊平,M,董事会成员,董事,博士,中国,1978,20170721,None
1,000058.SZ,20190925,张良,M,董事会成员,董事,硕士,中国,1971,20180329,None
2,000058.SZ,20190925,张宇杰,M,监事,监事,硕士,中国,1963,20180329,None
3,000058.SZ,20190925,高建柏,M,董事会成员,董事,硕士,中国,1964,20180913,None
4,000058.SZ,20190925,李立武,M,董事会成员,董事,本科,中国,1980,20190423,None


In [70]:
len(managers_df)

150842

In [71]:
def append_date(field: str):
    def func(x):
        f_str = x[field]
        if f_str is None:
            return None
        if len(f_str) == 4:
            return f_str + '0101'
        elif len(f_str) == 6:
            return f_str + '01'
        elif len(f_str) == 8:
            return f_str
        else:
            return None
    return func

In [72]:
managers_df['begin_date'] = managers_df.apply(append_date('begin_date'), axis=1)

In [73]:
managers_df['end_date'] = managers_df.apply(append_date('end_date'), axis=1)

In [74]:
managers_df['begin_date'] = pd.to_datetime(managers_df['begin_date'])

In [75]:
managers_df['end_date'] = pd.to_datetime(managers_df['end_date'])

In [76]:
managers_df.head()

,ts_code,ann_date,name,gender,lev,title,edu,national,birthday,begin_date,end_date
0,000058.SZ,20190925,徐腊平,M,董事会成员,董事,博士,中国,1978,2017-07-21,NaT
1,000058.SZ,20190925,张良,M,董事会成员,董事,硕士,中国,1971,2018-03-29,NaT
2,000058.SZ,20190925,张宇杰,M,监事,监事,硕士,中国,1963,2018-03-29,NaT
3,000058.SZ,20190925,高建柏,M,董事会成员,董事,硕士,中国,1964,2018-09-13,NaT
4,000058.SZ,20190925,李立武,M,董事会成员,董事,本科,中国,1980,2019-04-23,NaT


In [77]:
only_man_df = managers_df[['name', 'gender', 'edu', 'birthday']]

In [78]:
len(only_man_df)

150842

In [79]:
only_man_df = only_man_df.drop_duplicates()

In [80]:
len(only_man_df)

59929

In [81]:
only_man_df = only_man_df.reset_index(drop=True)

In [82]:
only_man_df['managerId'] = only_man_df.index

In [83]:
only_man_df.head()

,name,gender,edu,birthday,managerId
0,徐腊平,M,博士,1978,0
1,张良,M,硕士,1971,1
2,张宇杰,M,硕士,1963,2
3,高建柏,M,硕士,1964,3
4,李立武,M,本科,1980,4


In [84]:
only_man_df.describe()

,managerId
count,59929.000000
mean,29964.000000
std,17300.156478
min,0.000000
25%,14982.000000
50%,29964.000000
75%,44946.000000
max,59928.000000


In [85]:
man2code = {
    (row['name'], row['gender'], row['edu'], row['birthday']): row['managerId']
    for i, row in only_man_df.iterrows()
}

In [86]:
def build_relation(row):
    code = man2code[row['name'], row['gender'], row['edu'], row['birthday']]
    return [row['ts_code'], code, row['title'], row['begin_date'], row['end_date']]

In [87]:
relation_list = managers_df.apply(build_relation, axis=1)

In [88]:
relation_list = [x for x in relation_list]

In [89]:
relation_df = pd.DataFrame(relation_list, columns=[':END_ID(Company)', ':START_ID(Manager)', 'title:string', 'beginDate:date', 'endDate:date'])

In [90]:
only_man_df.columns = ['name:string', 'gender:string', 'edu:string', 'birthday:long', 'managerId:ID(Manager)']

In [91]:
only_man_df[':LABEL'] = 'Manager'
relation_df[':TYPE'] = 'work_in'

In [92]:
only_man_df.head()

,name:string,gender:string,edu:string,birthday:long,managerId:ID(Manager),:LABEL
0,徐腊平,M,博士,1978,0,Manager
1,张良,M,硕士,1971,1,Manager
2,张宇杰,M,硕士,1963,2,Manager
3,高建柏,M,硕士,1964,3,Manager
4,李立武,M,本科,1980,4,Manager


In [93]:
relation_df.head()

,:END_ID(Company),:START_ID(Manager),title:string,beginDate:date,endDate:date,:TYPE
0,000058.SZ,0,董事,2017-07-21,NaT,work_in
1,000058.SZ,1,董事,2018-03-29,NaT,work_in
2,000058.SZ,2,监事,2018-03-29,NaT,work_in
3,000058.SZ,3,董事,2018-09-13,NaT,work_in
4,000058.SZ,4,董事,2019-04-23,NaT,work_in


In [94]:
only_man_df.to_csv('import/manager.csv', index=False)
relation_df.to_csv('import/manager_company.csv', index=False)